In [297]:
from tensorflow.examples.tutorials.mnist import input_data

In [298]:
import numpy as np
import timeit

In [299]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [300]:
import tensorflow as tf

In [301]:
sess = tf.InteractiveSession()

In [302]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [303]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# W = tf.cast(W, tf.float32)
# b = tf.cast(b, tf.float32)

In [304]:
sess.run(tf.global_variables_initializer())

In [305]:
y = tf.matmul(x, W) + b

In [306]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [307]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [308]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    combine = []
    for i in range(len(batch[0])):
        halved = halved_array(batch[0][i])
        combine.append(halved)
    combined = np.asarray(combine) 
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [309]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.arg_max(y_, 1))

In [310]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [311]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9175


In [312]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [313]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    

In [314]:
#number of filters(32) are usually chosen as powers of 2
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [315]:
x_image = tf.reshape(x, [-1,28,28,1])

In [316]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [317]:
h_pool1 = max_pool_2x2(h_conv1)

In [318]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = weight_variable([64])

In [319]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [320]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

In [321]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [322]:
keep_prob = tf.placeholder(tf.float32)

In [323]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [324]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [325]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [326]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y_conv))

In [327]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [328]:
sess.run(tf.global_variables_initializer())

In [329]:
start_time = timeit.default_timer()
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
#     combine = []
#     for j in range(len(batch[0])):
#         halved = halved_array(batch[0][j])
#         combine.append(halved)
#     combined = np.asarray(combine) 
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
#         x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

print "Execution time: %f"%( timeit.default_timer() - start_time)

step 0, training accuracy 0.06
step 100, training accuracy 0.1
step 200, training accuracy 0.14
step 300, training accuracy 0.08
step 400, training accuracy 0.04
step 500, training accuracy 0.12
step 600, training accuracy 0.22
step 700, training accuracy 0.2
step 800, training accuracy 0.24
step 900, training accuracy 0.2
step 1000, training accuracy 0.18
step 1100, training accuracy 0.28
step 1200, training accuracy 0.26
step 1300, training accuracy 0.3
step 1400, training accuracy 0.32
step 1500, training accuracy 0.34
step 1600, training accuracy 0.44
step 1700, training accuracy 0.34
step 1800, training accuracy 0.44
step 1900, training accuracy 0.42
step 2000, training accuracy 0.3
step 2100, training accuracy 0.28
step 2200, training accuracy 0.46
step 2300, training accuracy 0.38
step 2400, training accuracy 0.48
step 2500, training accuracy 0.5
step 2600, training accuracy 0.52
step 2700, training accuracy 0.6
step 2800, training accuracy 0.44
step 2900, training accuracy 0.64

In [83]:
from PIL import Image
import numpy as np
w, h= 512, 512
data = np.zeros((h, w, 3), dtype=np.uint8)
data[256, 256] = [0, 255, 0]
img = Image.fromarray(data, 'RGB')
img.show()

In [84]:
digit = mnist.train.images[1004]
digit = digit.reshape(28,28)

digit = halved_array(digit)

NameError: name 'halved_array' is not defined

In [99]:
def halved_array(array):
    reshaped = array.reshape(28,28)
    ones = np.ones([14,28])
    zeros = np.zeros([14,28])
    combined = np.stack((ones, zeros), axis=0)
    combined = combined.reshape(28,28)
    fill_with_zeros = np.multiply(reshaped, combined)
    return fill_with_zeros.reshape(784)

In [ ]:
import scipy.misc
scipy.misc.imsave('outfile.jpg', digit)

In [ ]:
length = len(mnist.train.images)
halved = np.zeros(shape=(55000, 784))
for i in range(length):
    halved[i] = halved_array(mnist.train.images[i])

In [118]:
batch = mnist.train.next_batch(100)
images = batch[0]

In [122]:
combine = []
for i in range(len(images)):
    halved = halved_array(images[i])
    combine.append(halved)
combine = np.asarray(combine)    

In [121]:
combine.shape

(100, 784)

(100, 784)